In [5]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 3
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 3,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "code": {
            "font-size": 2,
        },
        "pre": {
            "font-size": 2,
        }
    }
});

<IPython.core.display.Javascript object>

In [6]:
print windowSize

(1303, 838)


# Finding TFBS Motifs in our Lifetime

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/LifetimeAchievement.jpg" width="300" style="float: right; clear: right; margin: 24px;">

* Recall from last time that the *Brute Force* approach for finding a common 10-mer motif common to 10 sequences of length 80 bases was going to take up roughly 30,000 years
* Today well consider alternative and non-obvious approaches for solving this problem
* We will trade one old man (us) for another (an Oracle)

<p style="text-align: right; clear: right;">1</p>

# Recall from last Lecture

* The following set of 10 sequences have an embedded noisy motif, *TAGATCCGAA*.

<pre><code>
 1 tagtggtcttttgagtg<span style="color: blue; font-weight: bold;">TAGATC<span style="color: red; font-weight: bold;">T</span>GAA</span>gggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat   TAGATCTGAA
 2 cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagt<span style="color: blue; font-weight: bold;">T<span style="color: red; font-weight: bold;">G</span>GATCCGAA</span>actggagtttaatcggagtcctt   TGGATCCGAA
 3 gttacttgtgagcctggt<span style="color: blue; font-weight: bold;">TAGA<span style="color: red; font-weight: bold;">C</span>CCGAA</span>atataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt   TAGACCCGAA
 4 aacatcaggctttgattaaacaatttaagcacg<span style="color: blue; font-weight: bold;">TA<span style="color: red; font-weight: bold;">A</span>ATCCGAA</span>ttgacctgatgacaatacggaacatgccggctccggg   TAAATCCGAA
 5 accaccggataggctgcttat<span style="color: blue; font-weight: bold;">TAG<span style="color: red; font-weight: bold;">G</span>TCC<span style="color: red; font-weight: bold;">A</span>AA</span>aggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac   TAGGTCCAAA
 6 <span style="color: blue; font-weight: bold;">TAGAT<span style="color: red; font-weight: bold;">T</span>CGAA</span>tcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc   TAGATTCGAA
 7 gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtg<span style="color: blue; font-weight: bold;"><span style="color: red; font-weight: bold;">C</span>AGATCCGAA</span>cgtctctggaggggtcgtgcgcta   CAGATCCGAA
 8 atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtg<span style="color: blue; font-weight: bold;">TAGATCCG<span style="color: red; font-weight: bold;">T</span>A</span>   TAGATCCGTA
 9 ttcttacacccttctt<span style="color: blue; font-weight: bold;">TAGATCC<span style="color: red; font-weight: bold;">A</span>AA</span>cctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac   TAGATCCAAA
10 ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacagg<span style="color: blue; font-weight: bold;">T<span style="color: red; font-weight: bold;">C</span>GATCCGAA</span>attcg   TCGATCCGAA
                                                                                      9+9+9+9+9
                                                                                      +8+9+9+8+10
                                                                                      = 89
</code></pre>

Some notes:
1. There are no exact matches
2. The consensus motif gives a good score  

<p style="text-align: right; clear: right;">2</p>

# Consensus Scoring Function
<div>
&bullet;&nbsp;We developed an $O(k)$ consensus scoring function to address noise (inexact matches)<br>
&bullet;&nbsp;But, we need to apply it an exponential number, $O(N^M)$ of times!<br>
&bullet;&nbsp;Here's the scoring function...
</div>

In [7]:
def Score(s, DNA, k):
    """ 
        compute the consensus SCORE of a given k-mer 
        alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore
            DNA = list of nucleotide strings
            k = Target Motif length
    """
    score = 0
    for i in xrange(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        score += max(cnt.itervalues())
    return score

<p style="text-align: right; clear: right;">3</p>

# And here's the Score we're looking for...

In [8]:
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']

In [9]:
print Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10)

89


In [13]:
%timeit Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10)

10000 loops, best of 3: 40 µs per loop


So even at a blazing $40 \mu s$ we'll need many lifetimes to compute the $70^{10}$ scores 

<p style="text-align: right; clear: right;">4</p>

# Pruning Trees
<div style="font-size: 80%;">
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/PruneTrees.png" width="160" style="float: right; clear: right; margin: 24px;">
&bullet;&nbsp;One method for reducing the computational cost of a search algorithm is to *prune* the space of permutations that could not possibly lead to a better answer than the current best answer.

&bullet;&nbsp;Pruning decisions are based on ***solutions to subproblems*** that appear early on and offer no hope

&bullet;&nbsp;How does this apply to our Motif finding problem?

Consider any permutation of offsets that begins with the indices [25, 63, 10, 43, ....]. Just based on the first 4 indices the largest possible score is 17 + 6*10 = 87, which assumes that all 6 remaining strings match perfectly at all 10 positions.

<pre style="color: #000080; font-weight: bold;">
 DNA[0][25:35]        a  a  g  g  g  a  a  a  g  t
 DNA[1][63:73]        g  t  t  t  a  a  t  c  g  g
 DNA[2][10:20]        a  g  c  c  t  g  g  t  t  a
 DNA[3][43:53]        t  t  g  a  c  c  t  g  a  t
                   a [2, 1, 0, 1, 1, 2, 1, 1, 1, 1]
       Profile     c [0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
                   g [1, 1, 2, 1, 1, 1, 1, 1, 2, 1]
                   t [1, 2, 1, 1, 1, 0, 2, 1, 1, 2]
                     [2, 2, 2, 1, 1, 2, 2, 1, 2, 2] Score = 17
</pre>

If the best answer so far is 89, there is no need to consider the 70<sup>6</sup> offset permuations that start with these 4 indices.
</div>
<p style="text-align: right; clear: right;">5</p>

# Search Trees

* Our standard method for enumerating permutations can be considered as a traversal of leaf nodes in a search tree
* Suppose after checking the first few offsets could know already that *any* score of children nodes could not beat the best score seen so far?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/SearchTree.png" width="600" class="centeredImg">

<p style="text-align: right; clear: right;">6</p>

# Branch-and-Bound Motif Search

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/TreeTrim.png" width="400" style="float: right; clear: right; margin: 24px;">

* Since each level of the tree goes deeper into search, discarding a prefix discards all following branches
* This saves us from looking at $(N – k + 1)^{M-depth}$ leaves
* Note our enumeration of tree-branches is *depth-first*
* We'll formulate of trimming algorithm as a recursive algorithm

<p style="text-align: right; clear: right;">7</p>

# A Recursive Exploration of a Search Tree

In [14]:
bestAlignment = []
prunedPaths = 0

def exploreMotifs(DNA,k,path,bestScore):
    """ Search for a k-length motif in the list of DNA sequences by exploring
        all paths in a search tree. Each call extends path by one. Once the
        path reaches the number of DNA strings a score is computed. """
    global bestAlignment, prunedPaths
    depth = len(path)
    M = len(DNA)
    if (depth == M):            # here we have an index in all M sequences
        s = Score(path,DNA,k)
        if (s > bestScore):
            bestAlignment = [p for p in path]
            return s
        else:
            return bestScore
    else:
        # Let's consider if an optimistic best score can beat the best score so far
        if (depth > 1):
            OptimisticScore = k*(M-depth) + Score(path,DNA,k)
        else:
            OptimisticScore = k*M
        if (OptimisticScore < bestScore):
            prunedPaths = prunedPaths + 1
            return bestScore
        else:
            for s in xrange(len(DNA[depth])-k+1):
                newPath = tuple([i for i in path] + [s])
                bestScore = exploreMotifs(DNA,k,newPath,bestScore)
            return bestScore

def BranchAndBoundMotifSearch(DNA, k):
    """ Finds a k-length motif within a list of DNA sequences"""
    global bestAlignment, prunedPaths
    bestAlignment = []
    prunedPaths = 0
    bestScore = 0
    bestScore = exploreMotifs(DNA,k,[],bestScore)
    print bestAlignment, bestScore, prunedPaths

%time BranchAndBoundMotifSearch(seqApprox[0:6], 10)

[17, 47, 18, 33, 21, 0] 53 8615931
CPU times: user 5min 17s, sys: 713 ms, total: 5min 18s
Wall time: 5min 17s


<p style="text-align: right; clear: right;">8</p>

# Observations

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/AWinner.png" width="100" style="float: right; clear: right; margin: 24px;">

* For our problem instance, Branch-and-Bound Motif finding is significantly faster
  - It found a motif in the first 6 strings in less time than the Brute Force approach found a solution in the first 4 strings
  - More than $70^2 \approx 5000$ times faster
  - It did so by trimming more than 8 Million paths
  - Trimming added extra calls to Score (no worse than doubling the worst-case number of calls), but ended up saving even more *hopeless* calls along longer paths.
  - In practice, Branch-and-Bound, significantly improved the average performance 
* Does this improve the worst-case performance from $O(kN^M)$?
  - What if all of our motifs were found at the end of each DNA string?
  - How do we avoid these worse case data sets?
  - Randomize the search-tree tranversal order

<p style="text-align: right; clear: right;">9</p>


# We need a new approach

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/guru.jpg" width="400" style="float: right; clear: right; margin: 24px;">

* Enumerating every possible permuation of motif positions is *still* not getting us the speed we want.
* Let's try another tried-and-tested approach to algorithm design, *mixing up the problem*
  - Suppose that some *Oracle* could tell us what the motif is
  - How long would it take us to find its position in each string?
  - We could compute the Hamming Distance from our given motif to the k-mer at every position of each DNA sequence and keep track of the smallest distance and its position on each string.
  - These positions are our best guess of where the motif can be found on each string
* Let's call this approach *scanning-and-scoring* to find a given motif.

<p style="text-align: right; clear: right;">10</p>

# Scanning-and-Scoring a Motif 

In [19]:
import itertools

for pattern in itertools.product('acgt', repeat=3):
    print ''.join(pattern)


aaa
aac
aag
aat
aca
acc
acg
act
aga
agc
agg
agt
ata
atc
atg
att
caa
cac
cag
cat
cca
ccc
ccg
cct
cga
cgc
cgg
cgt
cta
ctc
ctg
ctt
gaa
gac
gag
gat
gca
gcc
gcg
gct
gga
ggc
ggg
ggt
gta
gtc
gtg
gtt
taa
tac
tag
tat
tca
tcc
tcg
tct
tga
tgc
tgg
tgt
tta
ttc
ttg
ttt


In [15]:
def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in xrange(len(seq)-k+1):
            HammingDist = sum([1 for i in xrange(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

In [16]:
print ScanAndScoreMotif(seqApprox, "tagatccgaa")
%timeit ScanAndScoreMotif(seqApprox, "tagatccgaa")

([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11)
1000 loops, best of 3: 1.4 ms per loop


##### Wow, we can test over 650 motifs per second!
<p style="text-align: right; clear: right;">11</p>

# Scan-and-Score Motif Performance

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/ConsensusDist.png" width="500" style="float: right; clear: right; margin: 16px;">

* There are $M (N - k + 1)$ positions to test the motif,<br> and each test requires $k$ tests.
* So each scan is $O(M N k)$.<br><br>
* So where where do we get candidate motifs?
* Can we try all of them? There are $4^{10} = 1048576$ in our example.
  - Do the math, 1048576 motifs &times; 2 mS  &approx; 35 mins
  - Not fast, but less than a lifetime<br><br>
* This approach is called a ***Median String Motif Search***
* Recall from last Lecture that a string that minimizes *Hamming distance* is like finding a middle or median string that is closer to all instances than the instances are to each other.

<p style="text-align: right; clear: right;">12</p>

# Let's Do It

In [32]:
import itertools

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('acgt', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

%time MedianStringMotifSearch(seqApprox,10)

CPU times: user 26min 35s, sys: 613 ms, total: 26min 35s
Wall time: 26min 35s


([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')

<span style="font-size: 80%;">Should we declare victory and move on? Do you find anything uncomfortable about an algorithm that requires $O(M N k 4^k)$ steps?</span>

<p style="text-align: right; clear: right;">13</p>

# Notes on Median String Motif Search
* Similarities between finding and alignment with minimal Hamming Distance and maximizing a Motif's consensus score.
* In fact, if instead of counting mismatches as in the code fragment:
     <pre>HammingDist = sum([1 for i in xrange(k) if motif[i] != seq[s+i]])</pre>
  we had counted matches
     <pre>Matches = sum([1 for i in xrange(k) if motif[i] == seq[s+i]])</pre>
  and found the *maximum(TotalMatches)* instead of the *min(TotalHammingDistance)* we would be using the same measure as *Score()*.
* Thus, we expect *MedianStringMotifSearch()* to give the same answer as either *BruteForceMotifSearch()* or *BranchAndBoundMotifSearch()*.
* However, the $4^k$ term raises some concerns. If k were instead 20, then we'd have to Scan-and-Score more than $10^{12}$ times. Another *not-in-a-lifetime* algorithm
* We can also apply the *Branch-and-Bound* approach to the Median string method, but, as before it would only improve the average case.
<p style="text-align: right; clear: right;">14</p>

# Other ways to guess the motif?

* If we *knew* that the motif that we are looking for was contained *somewhere* in our DNA sequences we could test the $(N-k+1)t$ motifs from our DNA, giving a $O(N^2t^2)$ algorithm.<br><br>
* Unfortunately, as you may recall ***our motif***&nbsp; did not appear actually appear in our data.<br><br>
* You could keep track of a few good *motif candidates* using a manageable and perhaps random subsets of the given DNA sequences, and use them as your candidate motifs.

<p style="text-align: right; clear: right;">15</p>

# Let's try considering only Motifs seen in the DNA

In [29]:
def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in xrange(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print "%d Motifs in our set" % len(motifSet)
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

%time ContainedMotifSearch(seqApprox,10)

709 Motifs in our set
CPU times: user 1.04 s, sys: 11 ms, total: 1.05 s
Wall time: 1.04 s


([17, 31, 18, 33, 21, 0, 46, 70, 16, 65], 17, 'tagatccaaa')

<span style="font-size: 80%;">Not exactly the motif we were looking for (off by a 'g'),
&nbsp;&nbsp;&nbsp;[17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatcc<span style="color: red;">g</span>aa',
but boy was it fast! Where's a good Oracle when you need one?</span>
<p style="text-align: right; clear: right;">16</p>

# Insights from the consensus score matrix

If we call Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10)

<pre style="color: #000080; font-weight: bold;">

DNA[0][17:27]    t  a  g  a  t  c  t  g  a  a
DNA[1][31:41]    t  a  g  a  c  c  a  a  a  a
DNA[2][18:28]    t  a  g  a  c  c  c  g  a  a
DNA[3][33:43]    t  a  a  a  t  c  c  g  a  a
DNA[4][21:31]    t  a  g  g  t  c  c  a  a  a
DNA[5][ 0:10]    t  a  g  a  t  t  c  g  a  a
DNA[6][46:56]    c  a  g  a  t  c  c  g  a  a
DNA[7][70:80]    t  a  g  a  t  c  c  g  t  a
DNA[8][16:26]    t  a  g  a  t  c  c  a  a  a
DNA[9][65:75]    t  c  g  a  t  c  c  g  a  a
              a [0, 9, 1, 9, 0, 0, 1, 3, 9,10]
              c [1, 1, 0, 0, 2, 9, 8, 0, 0, 0]
              g [0, 0, 9, 1, 0, 0, 0, 7, 0, 0]
              t [9, 0, 0, 0, 8, 1, 1, 0, 1, 0]
                [9, 9, 9, 9, 8, 9, 8, 7, 9,10]  Score = 87
Consensus        t  a  g  a  t  c  c  g  a  a   Our motif!
</pre>

Any Ideas?

<p style="text-align: right; clear: right;">17</p>

# Contained Consensus Motif Search

In [30]:
def Consensus(s, DNA, k):
    """ compute the consensus k-Motif of an alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
            k = Target Motif length """
    consensus = ''
    for i in xrange(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        consensus += max(cnt.iteritems(), key=lambda tup: tup[1])[0]
    return consensus

def ContainedConsensusMotifSearch(DNA,k):
    bestAlignment, minHammingDist, kmer = ContainedMotifSearch(DNA,k)
    motif = Consensus(bestAlignment,DNA,k)
    newAlignment, HammingDist = ScanAndScoreMotif(DNA, motif)
    return newAlignment, HammingDist, motif

%time ContainedConsensusMotifSearch(seqApprox,10)

709 Motifs in our set
CPU times: user 1.06 s, sys: 23 ms, total: 1.08 s
Wall time: 1.06 s


([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')

<p style="margin-left: 800px;">Now we're cooking!</p>

<p style="text-align: right; clear: right;">18</p>

# Dad, are we there yet?

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/AreWeThere.jpg" width="350" style="float: right; clear: right; margin: 16px;">

We got the answer that we were looking for, <span style="font-size: 200%;">but</span>

* How can we be sure it will always give the correct answer?
  - Our other methods were exhaustive, they examined every possibility
  - This method considers only a subset of solutions, picks the best one in a *greedy* fashion
  - What if there had been ties amoung the candidate motifs?
  - What if the consensus score (87% matches) had been lower
  - Would we, should we, be satisfied?<br><br>
* It's one thing to be greedy, and another to be both *greedy and biased*
    - Our method is greedy in that it considers only the best contained motif, greedy methods are subject to falling into *local minimums*
    - Since consider only subsequences as motifs we introduce bias<br><br>
* Note that Consensus can generate motifs not seen in our data

<p style="text-align: right; clear: right;">19</p>

# A randomized approach to motif finding

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/Randomized.png" width="300" style="float: right; clear: right; margin: 16px;">

* One way to avoid bias and local minima is to introduce *randomness*
* We can generate candidate motifs from our data by treating it as distribution
  - likely motif candidates from this distribution are those generated by Consensus
  - Consensus strings can be tested by Scan-and-Score and their alignments lead to new consensus strings
  - Eventually, we should converge to some local minimal answer
* To avoid finding a local minimum, we try several random starts, and search for the best score amongst all these starts.
* A randomized algorithm does not guarantee an optimal solution. Instead it promises a good/plausible answer on average, and it is not susceptible to a worse-case data sets as our greedy/biased method was.

<p style="text-align: right; clear: right;">20</p>

# Code for Randomized Motif Search

In [31]:
import random

def RandomizedMotifSearch(DNA,k):
    """ Searches for a k-length motif that appears 
    in all given DNA sequences. It begins with a
    random set of candidate consensus motifs 
    derived from the data. It refines the motif
    until a true consensus emerges."""
    
    # Seed motifs from random alignments
    motifSet = set()
    for i in xrange(500):
        randomAlignment = [random.randint(0,len(DNA[j])-k) for j in xrange(len(DNA))]
        motif = Consensus(randomAlignment, DNA, k)
        motifSet.add(motif)

    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    testSet = motifSet.copy()
    while (len(testSet) > 0):
        print len(motifSet),
        nextSet = set()
        for motif in testSet:
            align, dist = ScanAndScoreMotif(DNA, motif)
            # add new motifs based on these alignments
            newMotif = Consensus(align, DNA, k)
            if (newMotif not in motifSet):
                nextSet.add(newMotif)
            if (dist < minHammingDist):
                bestAlignment = [s for s in align]
                minHammingDist = dist
                kmer = motif
        testSet = nextSet.copy()
        motifSet = motifSet | nextSet
    return bestAlignment, minHammingDist, kmer

<p style="text-align: right; clear: right;">21</p>

# Let's try it

In [28]:
%time RandomizedMotifSearch(seqApprox,10)

500 749 822 839 842CPU times: user 1.43 s, sys: 23 ms, total: 1.45 s
Wall time: 1.56 s


([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')

<br>
Randomized algorithms should be restarted multiple times to insure a stable solution.

In [29]:
for i in xrange(10):
    print RandomizedMotifSearch(seqApprox,10)

500 751 820 836 837 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 750 825 838 844 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 755 837 856 859 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
499 745 814 831 834 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 760 837 859 862 863 864 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 744 813 825 827 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
498 746 830 846 850 851 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 766 848 864 866 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 728 800 810 811 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500 750 833 851 852 ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')


<p style="text-align: right; clear: right;">22</p>

# Lessons Learned

* We can find Motifs in our lifetime
  - Practical exhaustive search algorithm for small k, MedianStringMotifSearch()
  - Practical fast algorthim RandomizedMotifSearch(DNA,k)
* Three algorithm design approaches "Branch-and-Bound", "Greedy", and "Randomized"
* Reversing the objective, pretending that you know the answer, and validating it
* The power of randomness
  - Not susceptable to worse case data
  - Avoids local minimums that plague some greedy algorithms
<div align="center">
<img width="200px" src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/branch.png" style="display: inline;">
<img width="200px" src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/greedy.png" style="display: inline;">
<img width="200px" src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/random.png" style="display: inline;">
</div>
<p style="text-align: right; clear: right;">23</p>